# PR checks

In [1]:
import pennylane as qml

## PR 1

https://github.com/PennyLaneAI/pennylane/pull/5036

Checking docstrings:

In [2]:
from pennylane.qchem.openfermion_obs import dipole_of
from pennylane import numpy as np

symbols = ["H", "H", "H"]
coordinates = np.array([0.028, 0.054, 0.0, 0.986, 1.610, 0.0, 1.855, 0.002, 0.0])
dipole_obs = dipole_of(symbols, coordinates, charge=1)
print(dipole_obs)

[<Hamiltonian: terms=18, wires=[0, 1, 2, 3, 4, 5]>, <Hamiltonian: terms=18, wires=[0, 1, 2, 3, 4, 5]>, <Hamiltonian: terms=1, wires=[0]>]


In [3]:
print(dipole_obs[0])  # x-component of D

  (0.24190977644638015) [Z4]
+ (0.24190977644638015) [Z5]
+ (0.4781123173263871) [Z0]
+ (0.4781123173263871) [Z1]
+ (0.7144779061811476) [Z2]
+ (0.7144779061811476) [Z3]
+ (-0.39136384894889287) [Y0 Z1 Y2]
+ (-0.39136384894889287) [X0 Z1 X2]
+ (-0.39136384894889287) [Y1 Z2 Y3]
+ (-0.39136384894889287) [X1 Z2 X3]
+ (-0.11734958781015546) [Y2 Z3 Y4]
+ (-0.11734958781015546) [X2 Z3 X4]
+ (-0.11734958781015546) [Y3 Z4 Y5]
+ (-0.11734958781015546) [X3 Z4 X5]
+ (0.26611147045283673) [Y0 Z1 Z2 Z3 Y4]
+ (0.26611147045283673) [X0 Z1 Z2 Z3 X4]
+ (0.26611147045283673) [Y1 Z2 Z3 Z4 Y5]
+ (0.26611147045283673) [X1 Z2 Z3 Z4 X5]


Docstring are correctly rendered in documentation

## PR 2

https://github.com/PennyLaneAI/pennylane/pull/5083

This is simply a modification to the deprecations and removals guide. I think there is nothing to test.

I asked Matt and noticed where changes in the documentations are.

Docstring are correctly rendered in documentation

## PR 3

https://github.com/PennyLaneAI/pennylane/pull/5237

Checking docstrings:

In [4]:
dev = qml.device("default.qubit", shots=10)


@qml.qnode(dev, diff_method="parameter-shift")
def circuit(angle):
    qml.RX(angle, wires=0)
    return qml.sample(qml.PauliX(0))


angle = qml.numpy.array(0.1)
res = qml.jacobian(circuit)(angle)

In [5]:
dev = qml.device("default.qubit", shots=[(1, 10)])


@qml.qnode(dev, diff_method="parameter-shift")
def circuit(angle):
    qml.RX(angle, wires=0)
    return qml.expval(qml.PauliX(0))


def cost(angle):
    return qml.math.hstack(circuit(angle))


angle = qml.numpy.array(0.1)
res = qml.jacobian(cost)(angle)

Docstring are correctly rendered in documentation

## PR 4

https://github.com/PennyLaneAI/pennylane/pull/5080

Testing tests:

In [6]:
op = qml.CosineWindow([2, 1])
res = op.state_vector(wire_order=[0, 1, 2])
assert res.shape == (2, 2, 2)

expected_10 = qml.CosineWindow([0, 1]).state_vector(wire_order=[1, 0])
expected = np.stack([expected_10, np.zeros_like(expected_10)])
assert np.allclose(res, expected)

Everything seems fine. I could double check that the new code lines are actually executed, but I'll do that later if I have time

## PR 5

https://github.com/PennyLaneAI/pennylane/pull/5178

Checking the bug report to be sure it is fixed:

In [7]:
import tensorflow as tf


def grad(f):
    def wrapper(x):
        with tf.GradientTape() as tape:
            y = f(x)

        return tape.gradient(y, x)

    return wrapper


dev = qml.device("default.qubit")


@qml.qnode(dev)
def circuit(theta):
    meas = qml.iterative_qpe(qml.RZ(theta, wires=[0]), [1], iters=2)
    return qml.expval(meas[0])


@qml.qnode(dev)
def manual_circuit(phi):
    qml.Hadamard(wires=[1])
    qml.ctrl(qml.RZ(phi, wires=[0]) ** 2, control=[1])
    qml.Hadamard(wires=[1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 1])
    qml.Hadamard(wires=[1])
    qml.ctrl(qml.RZ(phi, wires=[0]), control=[1])
    qml.ctrl(qml.PhaseShift(-np.pi / 2, wires=[1]), control=[2])
    qml.Hadamard(wires=[1])
    qml.CNOT(wires=[1, 3])
    qml.CNOT(wires=[3, 1])

    return qml.expval(qml.Hermitian([[0, 0], [0, 1]], wires=3))


phi = tf.Variable(1.0)
assert np.isclose(grad(circuit)(phi), grad(manual_circuit)(phi))

2024-02-28 09:38:45.653246: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-28 09:38:45.679734: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 09:38:45.679758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 09:38:45.680470: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 09:38:45.685222: I tensorflow/core/platform/cpu_feature_guar

## PR 6

https://github.com/PennyLaneAI/pennylane/pull/5233

In [8]:
import pennylane as qml
from pennylane import numpy as pnp

import numpy as np


@qml.qnode(qml.device("default.qubit", wires=3), diff_method="adjoint")
def circuit(x):
    qml.RX(x, 0)
    qml.QubitUnitary(np.eye(8), [0, 1, 2])
    return qml.expval(qml.PauliZ(2))


x = pnp.array(1.1, requires_grad=True)
qml.jacobian(circuit)(x)

array(0.)

Solved: for this specific error there was an import error that needed to be changed: `numpy` had to be imported not from `pennylane`

## PR 7

https://github.com/PennyLaneAI/pennylane/pull/5091

This is just a changing in the doc. The modification is also present in the official documentation, so everything is fine.

## PR 8

https://github.com/PennyLaneAI/pennylane/pull/5047

Learned from Matt that I need to change to `latest` in the doc to see last modifications

## PR 9

https://github.com/PennyLaneAI/pennylane/pull/5031

Everything seems fine. 

By the way, this might be extremely useful for learning how to convert `Hamiltonian` and `Tensor` instances to `op_math`, since this is precisely what is happening in this PR   

## PR 10

https://github.com/PennyLaneAI/pennylane/pull/5001

Everything seems fine. 

This might be relevant for my current project (here it was implemented addition between `PauliWord` and `PauliSentence`)

In [9]:
XX = qml.pauli.pauli_arithmetic.PauliWord({0: "X", 1: "X"})
YY = qml.pauli.pauli_arithmetic.PauliWord({0: "Y", 1: "Y"})

H = 0.5 * XX + 0.7 * YY
isinstance(H, qml.pauli.pauli_arithmetic.PauliSentence)
True

True

## PR 11

https://github.com/PennyLaneAI/pennylane/pull/5003

Same as above. In this case, it was implemented subtraction to `PauliSentence` and `PauliWord`

## PR 12

https://github.com/PennyLaneAI/pennylane/pull/5018

In [10]:
X0 = qml.pauli.pauli_arithmetic.PauliWord({0: "X"})

np.allclose(qml.matrix(X0), X0.to_mat())

True

Everything seems good, but I left a note about some code lines not covered by tests

## PR 13

https://github.com/PennyLaneAI/pennylane/pull/5027

In [11]:
qml.dot([1.0, 1.0], [qml.PauliX(0), qml.PauliX(0)])

X(0) + X(0)

In [12]:
qml.dot([1.0, 1.0], [qml.PauliX(0), qml.PauliX(0)]).simplify()

2.0 * X(0)

Mai una gioia. C'era una modifica da fare nelle docstring ma pare che sia gia stata cambiata 5 giorni fa nel rc branch

## PR 14

https://github.com/PennyLaneAI/pennylane/pull/5026

In [13]:
import torch
import torch.nn as nn
import pennylane as qml
import pennylane.numpy as np


def QuantumLayer():
    n_qubits = 4
    dev = qml.device("default.qubit", wires=n_qubits)

    def _circuit(inputs, weights):
        qml.StatePrep(inputs, wires=[1, 2, 3])
        qml.RY(phi=weights, wires=[0])
        return qml.expval(qml.PauliZ(wires=0))

    qlayer = qml.QNode(_circuit, dev, interface="torch")
    weight_shapes = {"weights": (1)}
    return qml.qnn.TorchLayer(qlayer, weight_shapes)


class SimpleQuantumModel(nn.Module):
    def __init__(self):
        super(SimpleQuantumModel, self).__init__()
        self.quantum_layer = QuantumLayer()

    def forward(self, x):
        return self.quantum_layer(x)


model = SimpleQuantumModel()
features = torch.tensor(
    np.array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]),
    requires_grad=True,
)

print(model(features))

tensor([-0.2894, -0.2894], dtype=torch.float64, grad_fn=<ViewBackward0>)


## PR 15

https://github.com/PennyLaneAI/pennylane/pull/5137

Waiting for answer from Lillian (devo controllare la doc rendered by pennylane.templates)

In [14]:
coeffs = [lambda p, t: p * jnp.sin(t) for _ in range(2)]
ops = [qml.X(0), qml.Y(0)]
qml.dot(coeffs, ops)

(
    <lambda>(params_0, t) * X(0)
  + <lambda>(params_1, t) * Y(0)
)

## PR 16

https://github.com/PennyLaneAI/pennylane/pull/5215

Ho installato jax per testare le docstrings. Questo is the result, and everything seems fine to me:

In [17]:
coeffs = np.array([1.1, 2.2])
ops = [qml.X(0), qml.Y(0)]
qml.dot(coeffs, ops)

1.1 * X(0) + 2.2 * Y(0)

In [18]:
qml.dot(coeffs, ops, pauli=True)

1.1 * X(0)
+ 2.2 * Y(0)

In [19]:
qml.dot([1.0, 1.0], [qml.X(0), qml.X(0)])

X(0) + X(0)

In [20]:
qml.dot([1.0, 1.0], [qml.X(0), qml.X(0)]).simplify()

2.0 * X(0)

In [21]:
qml.dot([1, 2], [qml.X(0), qml.X(0)], pauli=True).operation()

3.0 * X(0)

In [22]:
import jax.numpy as jnp

coeffs = [lambda p, t: p * jnp.sin(t) for _ in range(2)]
ops = [qml.X(0), qml.Y(0)]
qml.dot(coeffs, ops)

(
    <lambda>(params_0, t) * X(0)
  + <lambda>(params_1, t) * Y(0)
)

In [23]:
import pennylane as qml
from jax import numpy as jnp

# defining the coefficients fj(p, t) for the two parametrized terms
f1 = lambda p, t: p * jnp.sin(t) * (t - 1)
f2 = lambda p, t: p[0] * jnp.cos(p[1] * t**2)

# defining the operations for the three terms in the Hamiltonian
XX = qml.X(0) @ qml.X(1)
YY = qml.Y(0) @ qml.Y(1)
ZZ = qml.Z(0) @ qml.Z(1)

In [24]:
# Option 1
H1 = 2 * XX + f1 * YY + f2 * ZZ

# Option 2
coeffs = [2, f1, f2]
ops = [XX, YY, ZZ]
H2 = qml.dot(coeffs, ops)

In [25]:
H1

(
    2 * X(0) @ X(1)
  + <lambda>(params_0, t) * Y(0) @ Y(1)
  + <lambda>(params_1, t) * Z(0) @ Z(1)
)

## PR 17

https://github.com/PennyLaneAI/pennylane/pull/5039

Tutto ok 

## PR 18

https://github.com/PennyLaneAI/pennylane/pull/5218

Tutto ok

## PR 19

https://github.com/PennyLaneAI/pennylane/pull/4857

Questa mi sembra solo una cosa che toglie alcune dipendenze

## PR 20

https://github.com/PennyLaneAI/pennylane/pull/5155

Questo is un bugfix. Risolve il problema in quanto:

In [16]:
import pennylane as qml
from pennylane import numpy as np

import tensorflow as tf

nqbits = 4
depth = 2

dev = qml.device("default.mixed", wires=nqbits)


@qml.qnode(dev)
def mitigated_qnode(inputs, weights):

    qml.AmplitudeEmbedding(features=inputs, wires=range(nqbits), normalize=True)
    qml.templates.StronglyEntanglingLayers(weights, wires=range(nqbits))

    return [qml.expval(qml.PauliZ(wires=i)) for i in range(nqbits)]


weight_shapes = {"weights": (depth, nqbits, 3)}
qlayer = qml.qnn.KerasLayer(mitigated_qnode, weight_shapes, output_dim=nqbits)

inputs = np.random.uniform(0, 1, size=(2**nqbits,), requires_grad=False)
inputs = tf.convert_to_tensor(inputs)

qlayer(inputs)

<tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 0.09469968,  0.01266057, -0.18701663,  0.3980241 ])>